In [15]:
from ase.io import read
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import os

from glob import glob
from mlptools.atoms.extract_atoms import extract_atoms

## LAMMPSのdumpファイルから入射直後の構造を取得する

In [16]:
path2dump = '/Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/sputtering/SiO2/tersoff/dump.lammpstrj'
# atoms = read(path2dump, index=0, format='lammps-dump-text')

In [17]:
from ovito.io import import_file
from ovito.io.ase import ovito_to_ase

# Import a sequence of files.
pipeline = import_file(path2dump)

In [18]:
import random
random.seed(1)

dump_structure_every_n_steps = 500
insert_atom_every_n_steps = 20000
n_frame_after_insertion = 3

insert_atom_every_n_frames = insert_atom_every_n_steps // dump_structure_every_n_steps
pipeline.source.num_frames, insert_atom_every_n_frames

insert_atom_frame_idx_list = [i * insert_atom_every_n_frames + 1 for i in range(pipeline.source.num_frames // insert_atom_every_n_frames)]
for i in range(1, n_frame_after_insertion):
    insert_atom_frame_idx_list_plus = [idx + i for idx in insert_atom_frame_idx_list]
    insert_atom_frame_idx_list += insert_atom_frame_idx_list_plus
insert_atom_frame_idx_list.sort()

# get some elements from list randomly
random_idx_list = random.sample(range(len(insert_atom_frame_idx_list)), 100)
random_idx_list.sort()
random_insert_atom_frame_idx_list = [insert_atom_frame_idx_list[idx] for idx in random_idx_list]
# ase_atoms_list = []

In [19]:
# Loop over all frames of the sequence.
path2save = "/Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/sputtering/SiO2/tersoff/atoms"

symbol_map = {
    "H": "Si",
    "He": "O",
    "Li": "Si"
}
for frame_index in random_insert_atom_frame_idx_list:
    filename = f'frame{frame_index}.pkl'
    data = pipeline.source.compute(frame_index)
    ase_atoms = ovito_to_ase(data)
    new_symbols = [symbol_map[symbol] for symbol in ase_atoms.get_chemical_symbols()]
    ase_atoms.set_chemical_symbols(new_symbols)

    print('Frame source:', data.attributes['SourceFile'])
    print('Frame index:', data.attributes['SourceFrame'])

    with open(os.path.join(path2save, filename), 'wb') as f:
        pickle.dump(ase_atoms, f)
    # break

Frame source: /Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/sputtering/SiO2/tersoff/dump.lammpstrj
Frame index: 1
Frame source: /Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/sputtering/SiO2/tersoff/dump.lammpstrj
Frame index: 2
Frame source: /Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/sputtering/SiO2/tersoff/dump.lammpstrj
Frame index: 3
Frame source: /Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/sputtering/SiO2/tersoff/dump.lammpstrj
Frame index: 42
Frame source: /Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/sputtering/SiO2/tersoff/dump.lammpstrj
Frame index: 43
Frame source: /Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/sputtering/SiO2/tersoff/dump.lammpstrj
Frame index: 44
Frame source: /Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/sputtering/SiO2/tersoff/dump.lammpstrj
Frame index: 81
Frame source: /Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/sputtering/SiO2/tersoff/dump.lammpstrj
Frame index: 161
Frame source: /Users/y1u0d2/Google Drive/マ

# ASE Atoms objectから特定の範囲の原子を取り出す

In [20]:
# load pickle file
path2atoms = "/Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/sputtering/SiO2/tersoff/atoms"
with open(os.path.join(path2atoms, 'frame802.pkl'), 'rb') as f:
    atoms = pickle.load(f)

In [49]:
atoms_position_df = pd.DataFrame(atoms.get_positions(), columns=['x', 'y', 'z'])
atoms_position_df['symbol'] = atoms.get_chemical_symbols()
atoms_position_df['size'] = 5

In [51]:
# plot 3D
import plotly.express as px
import seaborn as sns

coord = atoms.get_positions()
cell = atoms.get_cell()

fig = px.scatter_3d(atoms_position_df, x='x', y='y', z='z', color='symbol', size="size", size_max=3)
fig.show()

# fig = plt.figure(figsize=(10,6))
# ax = fig.add_subplot(111)

# sns.scatterplot(data=atoms_position_df, x='x', y='z', hue='symbol', ax=ax)
# ax.set_xlim(0, cell[0].max())
# ax.set_ylim(0, cell[2].max())
# plt.show()

# 指定の範囲で切り出す

In [58]:
from mlptools.atoms.extract_atoms import extract_atoms

# extract atoms on the given range
x_range = [10, 20]
y_range = [20, 30]
z_range = [77, 100]
extracted_atoms = extract_atoms(atoms, x_range, y_range, z_range)
print(f"Number of extracted atoms: {len(extracted_atoms)}")

extracted_atoms_position_df = pd.DataFrame(extracted_atoms.get_positions(), columns=['x', 'y', 'z'])
extracted_atoms_position_df['symbol'] = extracted_atoms.get_chemical_symbols()
extracted_atoms_position_df['size'] = 5

# plot 3D
fig = px.scatter_3d(extracted_atoms_position_df, x='x', y='y', z='z', color='symbol', size="size", size_max=5)
fig.show()

Number of extracted atoms: 58


# 全surfaceに対して切り出してデータを取得する

In [62]:
path2atoms = "/Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/sputtering/SiO2/tersoff/atoms"
path2save = "/Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/sputtering/SiO2/tersoff/surface_atoms"

atoms_path_list = glob(os.path.join(path2atoms, 'frame*.pkl'))
print(f"Number of atoms: {len(atoms_path_list)}") 

Number of atoms: 100


In [63]:
x_range_list = [[0, 10], [10, 20], [20, 30], [30, 40]]
y_range_list = [[0, 10], [10, 20], [20, 30], [30, 40]]
z_range = [80, 100]

range_combination = []
for x_range in x_range_list:
    for y_range in y_range_list:
        range_combination.append([x_range, y_range, z_range])
print(f"Number of range combination: {len(range_combination)}")

def random_str(length):
    import random
    import string
    return ''.join(random.choice(string.ascii_letters) for i in range(length))

for atoms_path in atoms_path_list:
    atoms = pickle.load(open(atoms_path, 'rb'))
    for x_range, y_range, z_range in range_combination:
        print("*-" * 50)
        print(f"x_range: {x_range}, y_range: {y_range}, z_range: {z_range}")
        extracted_atoms = extract_atoms(atoms, x_range, y_range, z_range)
        print(f"Number of extracted atoms: {len(extracted_atoms)}")
        # save atoms
        filename = f'{random_str(50)}.pkl'
        with open(os.path.join(path2save, filename), 'wb') as f:
            pickle.dump(extracted_atoms, f)
        print(f"Saved: {filename}")
        print("*-" * 50)

Number of range combination: 16
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
x_range: [0, 10], y_range: [0, 10], z_range: [80, 100]
Number of extracted atoms: 40
Saved: nHnpvreeSHQxDGJVdktPVTJrwNVozJzlEYqNvToqNTpQbNzuBW.pkl
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
x_range: [0, 10], y_range: [10, 20], z_range: [80, 100]
Number of extracted atoms: 36
Saved: pYrmeOUkLCLUjMqDHkiXiTCxtWzphTnTRtegozuFglcdZMbWnR.pkl
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
x_range: [0, 10], y_range: [20, 30], z_range: [80, 100]
Number of extracted atoms: 38
Saved: cFTHUNCvQrhNSlgozoFCyWkopsDJLynCTqvFLhnfcaZaEuyLsm.pkl
*-*-*-*-*-*-*-*-*-*-